பின்வரும் நோட்புக்குகளை இயக்க, நீங்கள் இன்னும் செய்யவில்லை என்றால், .env கோப்பில் `OPENAI_API_KEY` என openai விசையை அமைக்க வேண்டும்.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

அடுத்து, நாம் Embedding Index ஐ Pandas Dataframe இல் ஏற்றப்போகிறோம். Embedding Index `embedding_index_3m.json` என்ற JSON கோப்பில் சேமிக்கப்பட்டுள்ளது. Embedding Index இல் 2023 அக்டோபர் இறுதிவரை உள்ள YouTube உரையாடல்களின் Embeddings உள்ளன.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

அடுத்து, `get_videos` என்ற ஒரு செயல்பாட்டை உருவாக்கப் போகிறோம், இது கேள்விக்கான Embedding Index-ஐ தேடும். இந்த செயல்பாடு கேள்விக்கான மிக ஒத்த 5 வீடியோக்களை திருப்பி வழங்கும். செயல்பாடு பின்வருமாறு செயல்படும்:

1. முதலில், Embedding Index-ஐ ஒரு நகலாக உருவாக்கப்படுகிறது.
2. அடுத்து, OpenAI Embedding API-ஐ பயன்படுத்தி கேள்விக்கான Embedding கணக்கிடப்படுகிறது.
3. பின்னர், Embedding Index-இல் `similarity` என்ற புதிய நெடுவரிசை உருவாக்கப்படுகிறது. `similarity` நெடுவரிசையில் கேள்வி Embedding மற்றும் ஒவ்வொரு வீடியோ பகுதியின் Embedding இடையேயான கோசைன் ஒத்திசைவு உள்ளது.
4. அடுத்து, `similarity` நெடுவரிசை மூலம் Embedding Index வடிகட்டப்படுகிறது. கோசைன் ஒத்திசைவு 0.75 அல்லது அதற்கு மேல் உள்ள வீடியோக்களை மட்டும் உள்ளடக்கியவாறு வடிகட்டப்படுகிறது.
5. இறுதியில், `similarity` நெடுவரிசை மூலம் Embedding Index வரிசைப்படுத்தப்பட்டு, மேல் 5 வீடியோக்கள் திருப்பி வழங்கப்படுகின்றன.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

இந்த செயல்பாடு மிகவும் எளிமையானது, இது தேடல் கேள்வியின் முடிவுகளை மட்டும் அச்சிடுகிறது.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. முதலில், Embedding Index ஒரு Pandas Dataframe ஆக ஏற்றப்படுகிறது.
2. அடுத்து, பயனரிடம் ஒரு கேள்வி உள்ளிடுமாறு கேட்கப்படுகிறது.
3. பின்னர், `get_videos` செயல்பாடு கேள்விக்கான Embedding Index ஐ தேட அழைக்கப்படுகிறது.
4. கடைசியாக, `display_results` செயல்பாடு பயனருக்கு முடிவுகளை காட்சிப்படுத்த அழைக்கப்படுகிறது.
5. பின்னர் பயனரிடம் மற்றொரு கேள்வி உள்ளிடுமாறு கேட்கப்படுகிறது. பயனர் `exit` என உள்ளிடும் வரை இந்த செயல்முறை தொடர்கிறது.

![](../../../../translated_images/ta/notebook-search.1e320b9c7fcbb0bc.png)

நீங்கள் ஒரு கேள்வி உள்ளிடுமாறு கேட்கப்படுவீர்கள். ஒரு கேள்வி உள்ளிட்டு enter அழுத்தவும். பயன்பாடு கேள்விக்கு தொடர்புடைய வீடியோக்களின் பட்டியலை வழங்கும். கேள்விக்கான பதில் வீடியோவில் எங்கு உள்ளது என்பதற்கான இணைப்பையும் பயன்பாடு வழங்கும்.

சோதிக்க சில கேள்விகள் இங்கே:

- Azure Machine Learning என்றால் என்ன?
- Convolutional neural networks எப்படி செயல்படுகின்றன?
- Neural network என்றால் என்ன?
- Azure Machine Learning உடன் Jupyter Notebooks பயன்படுத்தலாமா?
- ONNX என்றால் என்ன?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**குறிப்பு**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சித்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனிக்கவும். அசல் ஆவணம் அதன் சொந்த மொழியில் அதிகாரப்பூர்வ மூலமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
